# Создадим корпус текстов с покупками каждого клиента. Где каждый документ соответсвует одному клиенту, а каждое слово соответсвует одному товару. Применим TfidfVectorizer и получим признаки клиента

###  Импорт библиотек

In [145]:
import numpy as np
import pandas as pd
import csv
from datetime import datetime
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer

### Используем генератор строк

In [2]:
# Создаем генератор строк
def generator_of_rows(filename):
    '''
    Generator of rows in csv file
    Returns row per call.
    
    '''
    # Открываем файл
    with open(filename, "r") as csvfile:
        datareader = csv.reader(csvfile)
        for row in datareader:
            yield row

In [3]:
PATH='Data'
filename = PATH + '/purchases.csv'

### Получим нужный датафрейм

In [14]:
%%time
client = []
product = []
id_row = 0 
for row in generator_of_rows(filename):
    client.append(row[0])
    product.append(row[9])
    id_row += 1

Wall time: 1min 33s


Инициализируем начальный фрейм

In [20]:
product_df = pd.DataFrame()
product_df['client_id'] = client[1:]
product_df['product_id'] = product[1:]

Создаем словарь клиентов и переписываем client_id в более компактный вид - порядковый 1, 2, 3,...

In [90]:
set_clients = set(product_df['client_id'])
dict_client = dict.fromkeys(set_clients, 0)
new_id=1
for client in set_clients:
    dict_client[client] = new_id
    new_id+=1

Создаем словарь продуктов и переписываем product_id в более компактный вид - порядковый 1, 2, 3,...

In [54]:
set_products = set(product_df['product_id'])
dict_products = dict.fromkeys(set_products, 0)
new_id=1
for product in set_products:
    dict_products[product] = new_id
    new_id+=1

Записываем новый датафрейм с новыми id

In [93]:
product_df['product_id_new'] = [dict_products[product] for product in product_df['product_id']]
product_df['client_id_new'] = [dict_client[client] for client in product_df['client_id']]

client_product_df = product_df[['client_id_new','product_id_new']]

Индексируем по client_id_new

In [94]:
client_product_df.index = client_product_df['client_id_new']
client_product_df.drop(columns='client_id_new', inplace=True)

Сохраняем

In [96]:
client_product_df.to_csv('client_product.csv')

### Создаем корпус документов

Сначала корпус листов с интами

In [122]:
%%time
document = []
client_id = 0
id_row = 0
# Последнего выкидываем
for client in client_product_df.index.unique()[:-1]:
    text = []
    while client_product_df.index[id_row]==client:
        text.append(client_product_df['product_id_new'].iloc[id_row])
        id_row+=1
        
    document.append(text)

Wall time: 6min 30s


Затем в текст

In [142]:
%%time
text_document = []
for text in document:
    text_document.append(' '.join([str(word) for word in text]))

Wall time: 21.4 s


Сохранемя в pkl формате (для сохранения корпуса с интами может не хватить оперативки)

In [143]:
# with open("products_list.txt", "wb") as products_list:   #Pickling
#     pickle.dump(document, products_list)

with open("products_corpus.txt", "wb") as products_corp:   #Pickling
    pickle.dump(text_document, products_corp)   

if 'text_document' not in locals():
    with open("products_corpus.txt", "rb") as products_corp:   # Unpickling
        document = pickle.load(products_corp)

if 'document' not in locals():
    with open("products_list.txt", "rb") as products_list:   # Unpickling
        document = pickle.load(products_list)


Получем корпус длинной по количеству клиентов

In [139]:
len(text_document)

400161

In [146]:
text_document[0]

'5652 31162 23186 27164 20645 31560 21289 23146 9561 27574 40375 8509 18966 32236 39364 17029 27174 30207 23709 20099 21611 7018 41000 14281 51 5288 34273 37605 12418 37700 3603 34747 9766 17029 12349 7262 14281 40925 12824 28148 3836 4196 15306 19059 27079 21896 10265 19059 33177 5288 51 23146'

### Собираем vectorizer

Задаем параметры

In [129]:
vectorizer_params={'ngram_range': (1, 5), 
                       'max_features': 50000}
vectorizer = TfidfVectorizer(**vectorizer_params)

Строим признаки **(не хватает оперативки!!!)**

In [144]:
%%time
product_df_transform = vectorizer.fit_transform(text_document)

MemoryError: 